# Pulls data from the Copernicus Data Provider (European Commision) available at cds.climate.copernicus.eu
The output is a csv file containing the requested time series.

As currently downloading the complete (daily) data set is not supported due to size limitations - only sampled data for every 10th day is available

To use the data set legally it is required to create an account here: https://cds.climate.copernicus.eu/
Please then obtain your API key from your user provide and provide it as a parameter to this component

WARNING: This component currently only supports local execution (not Kubeflow/Airflow)

Future work  
[ ] Download complete data set by creating multile requests and then merge the results

In [ ]:
!pip3 install xarray==0.17.0 netcdf4==1.5.6 cdsapi==0.5.1 wget==3.2

In [ ]:
# @param api key in form UID:APIKey obtained from
# https://cds.climate.copernicus.eu/
# @param data_dir temporal data storage for local execution
# @param file_name csv file name

In [ ]:
import wget
wget.download(
    'https://raw.githubusercontent.com/elyra-ai/' +
    'component-library/master/claimed_utils.py'
)

In [2]:
import cdsapi
import xarray as xr
from claimed_utils import unzip
import pandas as pd
import os
import glob

In [3]:
apikey = os.environ.get('api key')
file_name = os.environ.get('file_name', 'data.csv')
data_dir = os.environ.get('data_dir', '../../data/')

In [4]:
skip = False

if os.path.exists(data_dir+file_name):
    skip = True

In [ ]:
if not skip:
    with open(os.path.expanduser('~/.cdsapirc'), "w") as myfile:
        myfile.write("url: https://cds.climate.copernicus.eu/api/v2\n")
        myfile.write("key: "+apikey+"\n")
        myfile.write("verify: 0")

In [ ]:
if not skip:
    
    c = cdsapi.Client()

    c.retrieve(
        'satellite-soil-moisture',
        {
            'variable': 'volumetric_surface_soil_moisture',
            'type_of_sensor': 'passive',
            'time_aggregation': 'month_average',
            'year': [
                '1978', '1979', '1980',
                '1981', '1982', '1983',
                '1984', '1985', '1986',
                '1987', '1988', '1989',
                '1990', '1991', '1992',
                '1993', '1994', '1995',
                '1996', '1997', '1998',
                '1999', '2000', '2001',
                '2002', '2003', '2004',
                '2005', '2006', '2007',
                '2008', '2009', '2010',
                '2011', '2012', '2013',
                '2014', '2015', '2016',
                '2017', '2018', '2019',
            ],
            'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
            ],
            'day': '01',
            'type_of_record': 'cdr',
            'version': 'v201912.0.0',
            'format': 'zip',
        },
        data_dir+'download.zip')

In [ ]:
if not skip:
    for f in glob.glob(data_dir+'*.nc'):
        os.remove(f)

In [ ]:
if not skip:
    unzip(data_dir, data_dir+'download.zip')

In [ ]:
skip = False

if not skip:
    for filename in os.listdir(data_dir):
        if filename.endswith(".nc") :
            print('Starting to process {}...'.format(filename))
            filename_csv = filename.split('.nc')[0]+'.csv'
            if not os.path.exists(data_dir+filename_csv):
                dset = xr.open_dataset(os.path.join(data_dir, filename))
                df = pd.DataFrame(dset['sm'].to_series())
                df.reset_index(inplace=True)
                df.to_csv(data_dir+filename_csv, index=False)
            else:
                print('CSV file {} already present, skipping...'.format(filename_csv))
        else:
            continue


In [ ]:
!echo "time,lat,lon,sm" > ../../data/data.csv
for file in `ls ../../data/C3S-SOILMOISTURE-*.csv`; do cat $file |tail -n +2 >> ../../data/data.csv; done